- ~~States~~
- Social Media used
- ~~Date~~ 
- ~~Money Mentioned~~ 
- ~~wfh~~
- community detection
- linguistic analysis
- Emails
- ~~URLS~~

In [2]:
import pandas as pd
import math
from ast import literal_eval

In [3]:
table = pd.read_csv("./csv.csv")
orig = pd.read_csv("../raw/usa_with_details.csv")
zipcodes = pd.read_csv("../csv/free-zipcode-database-Primary.csv") 
table_n = table.shape[0]

# todo : scams => company name , //orgs , ?// wfh single Node , //states , ? country names mentioned. 

In [15]:
# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process


In [54]:
def removeCommonWords(text):
    list_ = ["spoofer","spoof","imposter","impostor","poser"]
    for i in list_:
        text = text.replace(i,"")
    return text

In [55]:
companies = table['Company'].unique()
companies[28] =""
# sort them alphabetically and then take a closer look
companies.sort()
companies2 = []
for i in companies:
    l = i.lower().strip()
    l = removeCommonWords(l)
    companies2.append(l)


In [78]:
combined_list = companies2
cnt = 0
grs = list() # groups of names with distance > 80
label = []
for name in combined_list:
    for g in grs:
        if all(fuzz.ratio(name, w) > 80 for w in g):
            g.append(name)
            label.append(g[0])
            cnt+=1
            print(cnt)
            break
    else:
        grs.append([name, ])
        label.append(name)

print('output groups:', grs)
outlist = [el for g in grs for el in g]
print('output list:', outlist)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: 

In [22]:
loc_data ={}
for i in range(0,table_n):
    code_ = int(orig["p"][i])
    loc_data[code_] = zipcodes[zipcodes["Zipcode"]==code_][:].iloc(0)

In [23]:
table.columns
    

Index(['Unnamed: 0', 'Index', 'Company', 'Money', 'Date', 'Description',
       'WFH_job', 'MONEY', 'TITLE', 'ORGANIZATION', 'COUNTRY', 'DATE',
       'ORDINAL', 'LOCATION', 'NUMBER', 'DURATION', 'PERSON', 'MISC', 'CITY',
       'STATE_OR_PROVINCE', 'TIME', 'CRIMINAL_CHARGE', 'SET', 'EMAIL', 'URL',
       'HANDLE', 'CAUSE_OF_DEATH', 'NATIONALITY', 'PERCENT', 'RELIGION',
       'IDEOLOGY'],
      dtype='object')

In [37]:
orgs_dict = {}
orgs = []
rel_table_orgs = []

dates_dict = {}
dates = []

emails_dict = {}
emails = []
rel_table_emails = []

urls_dict = {}
urls = []
rel_table_urls = []

money_dict = {}
money_values = []
rel_table_money = []


new_table1 = []
for i in range (table_n) :
#     print(i)
    current = {}
    current["SCAMID"] = i
    current["SCAMNUMBER"] = i
    current["DESC"] = table["Description"][i]
    current["WFH"] = table["WFH_job"][i]
    x = table["Company"][i]
    
    # States
    code_ = int(orig["p"][i])
    if loc_data[code_][:].size == 0:
        continue
    current["STATE"] = str(loc_data[code_][0]["State"])
    
    # Dates
    date_dt1 = datetime.strptime(orig["c"][i],"%b %d, %Y")
    current["DATE"] = str(date_dt1.date())
    
    basic =  current["DATE"]
    if basic not in dates_dict:
        dates_dict[basic] = len(dates)
        dates.append({
            "ID" : len(dates),
            "DATE": basic
        })
    
    #URLS
    
    if isinstance(table["URL"][i],str) or not(math.isnan(table["URL"][i]) ):
        test_list = (literal_eval(table["URL"][i]))
        for url in test_list:
            url =  url.lower().strip()
            if url not in urls_dict:
                urls_dict[url] = len(urls)
                urls.append({
                    "ID": len(urls),
                    "URL":url
                })
            rel = {}
            rel["SCAMID"] = i
            rel["URL_ID"] = urls_dict[url]
            rel_table_urls.append(rel)  
    
    #Money
    
    if isinstance(table["MONEY"][i],str) or not(math.isnan(table["MONEY"][i]) ):
        test_list = (literal_eval(table["MONEY"][i]))
        for money in test_list:
            money =  money.lower().strip()
            if money not in money_dict:
                money_dict[money] = len(money_values)
                money_values.append({
                    "ID": len(money_values),
                    "MONEY_VALUE":money
                })
            rel = {}
            rel["SCAMID"] = i
            rel["MONEY_ID"] = money_dict[money]
            rel_table_money.append(rel)     
    
    # ORGS
    
    if isinstance(table["ORGANIZATION"][i],str) or not(math.isnan(table["ORGANIZATION"][i]) ):
        test_list = (literal_eval(table["ORGANIZATION"][i]))
        for org in test_list:
            org =  org.lower().strip()
            if org in orgs_dict:
                rel = {}
                rel["SCAMID"] = i
                rel["ORG_ID"] = orgs_dict[org]
            else :
                orgs_dict[org] = len(orgs)
                orgs.append({
                    "ID": len(orgs),
                    "ORG_NAME":org
                })
                rel = {}
                rel["SCAMID"] = i
                rel["ORG_ID"] = orgs_dict[org]
            rel_table_orgs.append(rel)
    new_table1.append(current)


'the sites moneyandtask.com and the monthlypayouts.com tell people to share their links and earn money for every lead sent them from link I got up to $955 before I asked for payout and never was given my payout they send you in circles with surveys they want you to do to prove you are not a robot but it is just a run around.'

In [69]:
print(rel_table_money[-1])
print(table["MONEY"][15799])
print(money_values[263])

{'SCAMID': 15799, 'MONEY_ID': 263}
['$3760', '$2700']
{'ID': 263, 'MONEY_VALUE': '$2700'}


In [71]:
new_table_frame = pd.DataFrame(rel_table_money) 
new_table_frame.to_csv("rel_money_scams.csv",index=False)

In [ ]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: -item[1])}

In [86]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["DESC"] = table["Description"][i]
    if isinstance(table["URL"][i],str) or not(math.isnan(table["URL"][i]) ):
        test_list = (literal_eval(table["URL"][i]))
        res = []
        [res.append(x) for x in test_list if x not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["URL"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1

In [87]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: -item[1])}

{'www.google.com/hangouts': 36,
 'Indeed.com': 30,
 'www.gmail.com': 27,
 'indeed.com': 21,
 'BBB.org': 10,
 'https://hangouts.google.com/': 10,
 'jobs.org': 9,
 'gmail.com': 8,
 'webstorejobs.org': 8,
 'https://hangouts.google.com': 5,
 '@gmail.com': 5,
 'ziprecruiter.com': 5,
 'care.com': 5,
 'www.usps.com': 5,
 'stockretail.org': 5,
 'mail.google.com': 4,
 'www.google.com/hangouts/': 4,
 'aol.com': 4,
 'bbb.org': 4,
 'https://docs.google.com/document/d/1ecGeM-NfJOAVv5Zm9xezteMdiPlei7VN8hN9tRawt88/': 4,
 'https://www.logistics-station.com/': 4,
 'http://www.gmail.com': 4,
 'careerbuilder.com': 4,
 'ZipRecruiter.com': 4,
 'www.adobe.com': 4,
 'monster.com': 4,
 'www.fedex.com': 3,
 '************@gmail.com': 3,
 'smtpsendemail.com': 3,
 'http://alpecourierservices.com/career': 3,
 'https://orders-crm.net': 3,
 'www.google.com/hangout': 3,
 'freelancer.com': 3,
 'http://www.google.com/hangouts/': 3,
 'retailpay.org': 3,
 'shopperweb.org': 3,
 'storejobs.org': 3,
 'http://www.pinnemploym

In [129]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["DESC"] = table["Description"][i]
    if isinstance(table["EMAIL"][i],str) or not(math.isnan(table["EMAIL"][i]) ):
        test_list = (literal_eval(table["EMAIL"][i]))
        res = []
        [res.append(x) for x in test_list if x not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["EMAIL"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1

In [130]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: -item[1])}

{'hr@manzanitapharmaceutical.com': 4,
 'brian.krum@xpress-shippingmail.com': 3,
 'Tereasa.Bur.sou@careerusahub.com': 2,
 'career@premierehubset.com': 2,
 'jobs@premierehubset.com': 2,
 'isaacdelphine568@gmail.com': 2,
 'hr@ewrix.com': 2,
 'hr@g-marketplace.com': 2,
 'kdavis@maylix.com': 2,
 'payroll@maylix.com': 2,
 'jamessurveygroups@gmail.com': 2,
 'contact@roundrockoqtel.com': 2,
 'HrCipherPharma@gmail.com': 2,
 'dr.francishorst@gmail.com': 2,
 'sarah.hassanein@xpress-shippingmail.com': 2,
 'nelson.ingram@xpress-shippingmail.com': 2,
 'benjaminchwartz@gmail.com': 2,
 'charlie@postalboardmail.com': 2,
 'ronald@postalboardmail.com': 2,
 'nick.tate@xpress-shippingmail.com': 2,
 'hiringdesk707@gmail.com': 2,
 'warrenfinegold290@gmail.com': 2,
 'scottwork300@gmail.com': 2,
 'Steve.brooks01@accountant.com': 2,
 'timwalk309@gmail.com': 2,
 'hr.ferrer.pharmaceutical@gmail.com': 2,
 'donna.powell@prime-craft.com': 2,
 'cbevaluationreport1@comcast.net': 2,
 'john@wal2walfunshop.com': 2,
 'cel

In [73]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["DESC"] = table["Description"][i]
    if isinstance(table["ORGANIZATION"][i],str) or not(math.isnan(table["ORGANIZATION"][i]) ):
        test_list = (literal_eval(table["ORGANIZATION"][i]))
        res = []
        [res.append(x) for x in test_list if x not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["ORGANIZATION"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1

In [74]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: -item[1])}

{'Google': 682,
 'Walmart': 380,
 'Amazon': 183,
 'Google Hangouts': 156,
 'Skype': 141,
 'Facebook': 132,
 'UPS': 131,
 'Apple': 120,
 'Customer Service': 120,
 'Microsoft': 119,
 'BBB': 111,
 'USPS': 109,
 'Bank': 99,
 'Company': 82,
 'Employee Wellness': 78,
 'FBI': 78,
 'Better Business Bureau': 77,
 'Fedex': 73,
 'AD&D Insurance': 71,
 'LLC': 69,
 'Western Union': 62,
 'Bank of America': 58,
 'CVS': 57,
 'Walgreens': 57,
 'Wal-Mart': 54,
 'Wells Fargo': 52,
 'PayPal': 51,
 'Kroger': 50,
 'FTC': 47,
 'Microsoft Office': 43,
 'Fed': 36,
 'Paypal': 31,
 'IRS': 29,
 'Ebay': 28,
 'Verizon': 28,
 'MoneyGram': 28,
 'HR Department': 25,
 'Moneygram': 24,
 'Fax Machine': 23,
 'AD & D Insurance': 22,
 'Linkedin': 21,
 'Peachtree Complete Accounting Software': 20,
 'USAA': 20,
 'eBay': 20,
 'Apple Store': 20,
 'Health Insurance': 19,
 'Macbook': 18,
 'Walgreen': 17,
 'DHL': 17,
 'Human Resources Department': 17,
 'Federal Trade Commission': 17,
 'Chase Bank': 17,
 'Kmart': 17,
 'Life Insuran

In [5]:
new_table2 = []
email_dict = {}
count = 0;
for i in range (table_n) :
    x = table["Company"][i]
    if isinstance(x,str):
        x = x.lower()
        if x not in email_dict:
            email_dict[x] = math.floor(count)
            count+=1
            new_table2.append({"SCAMMER": x})


In [126]:
new_table_frame = pd.DataFrame(new_table2) 
new_table_frame.to_csv("scammer.csv",index=True)

In [86]:
email_dict = {}
new_table1 = []
new_table2 = []
for i in range (table_n) :
    current = {}
    current["SCAMID"] = i
    current["DESC"] = table["Description"][i]
    if isinstance(table["COUNTRY"][i],str) or not(math.isnan(table["COUNTRY"][i]) ):
        test_list = (literal_eval(table["COUNTRY"][i]))
        res = []
        [res.append(x.lower()) for x in test_list if x.lower() not in res]
        for x in res:
            current1 = {}
            current1["SCAMID"] = i
            current1["COUNTRY"] = x
            new_table2.append(current1)
            if x in email_dict:
                email_dict[x] +=1
            else :
                email_dict[x] = 1

In [87]:
{k: v for k, v in sorted(email_dict.items(), key=lambda item: -item[1])}

{'us': 252,
 'usa': 150,
 'canada': 133,
 'united states': 124,
 'u.s.': 69,
 'america': 49,
 'uk': 37,
 'china': 35,
 'germany': 34,
 'the states': 34,
 'australia': 31,
 'india': 27,
 'united states of america': 23,
 'japan': 20,
 'united kingdom': 19,
 'italy': 15,
 'switzerland': 15,
 'ukraine': 14,
 'u.s.a.': 13,
 'france': 12,
 'sweden': 10,
 'u.s.a': 10,
 'england': 10,
 'singapore': 10,
 'russia': 9,
 'philippines': 8,
 'mexico': 7,
 'spain': 7,
 'poland': 7,
 'ph': 6,
 'denmark': 6,
 'turkey': 6,
 'austria': 5,
 'brazil': 4,
 'new zealand': 4,
 'south africa': 4,
 'panama': 4,
 'portugal': 3,
 'indonesia': 3,
 'norway': 3,
 'netherlands': 3,
 'nicaragua': 3,
 'ch': 3,
 'nigeria': 3,
 'jamaica': 3,
 'ireland': 3,
 'belgium': 3,
 'scotland united kingdom': 3,
 'colombia': 2,
 'namibia': 2,
 'malawi': 2,
 'iraq': 2,
 'uae': 2,
 'czech republic': 2,
 'cyprus': 2,
 'romania': 2,
 'venezuela': 2,
 'united arab emirates': 2,
 'kuwait': 2,
 'iceland': 2,
 'cuba': 1,
 'south korea': 1,

#### Dates


In [10]:
from datetime import datetime


In [18]:
dates_dict = {}
dates = []
for i in range (table_n) :
    date_dt1 = datetime.strptime(orig["c"][i],"%b %d, %Y")
    basic =  str(date_dt1.date())
    if basic not in dates_dict:
        dates_dict[basic] = len(dates_dict)
        dates.append({
            "ID" : len(dates_dict),
            "DATE": basic
        })

In [20]:
new_table_frame = pd.DataFrame(dates) 
new_table_frame.to_csv("dates.csv",index=False)